## 6_ovarlapping_removal
### Removes overlaps between OSM buildings and VIDA buildings directly in the DB2 database

### Initial configuration

#### To start working with this particular notebook, you need to provide necessary credential and settings
#### Below is an template of configuration, which is necessary prepare aside of this notebook and copy & paste all content in triple quotes to the next cell's input field

    """
    {
    "DB2_CONNECTION_STRING": "jdbc:db2://65beb513-5d3d-4101-9001-f42e9dc954b3.brt9d04f0cmqeb8u7740.databases.appdomain.cloud:30371/BLUDB:sslConnection=true;useJDBC4ColumnNameAndLabelSemantics=false;db2.jcc.charsetDecoderEncoder=3;",
    "DB2_USERNAME": "xxx",
    "DB2_PASSWORD": "xxx",
    "COUNTRY_TABLE": "FEATURES_DB_VIDA_EXTENDED",
    "AREA_THRESHOLD": 20
    }
    """

In [ ]:
# Read notebook configuration
import getpass
import json

config_str = getpass.getpass('Enter your prepared config: ')
config = json.loads(config_str)

In [ ]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd
import jaydebeapi as jdbc
import jpype
from tqdm import tqdm
import shapely
import os
from pyproj import Geod
from collections import Counter

geod = Geod(ellps="WGS84")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Switch between Kenya and India tables from the config and specify threshold
sql_tablename = config["COUNTRY_TABLE"]
area_threshold = config["AREA_THRESHOLD"]

In [ ]:
# connect to the IBM DB2 function
def connect_to_db():
    '''
        Connect to the IBM DB2 database
    '''
    
    jar = 'db2jcc4.jar'
    os.environ['CLASSPATH'] = jar

    args='-Djava.class.path=%s' % jar
    jvm_path = jpype.getDefaultJVMPath()
    try:
        jpype.startJVM(jvm_path, args)
    except Exception as e:
        print('startJVM exception: ', e)
        
    if jpype.isJVMStarted() and not jpype.isThreadAttachedToJVM():
        jpype.attachThreadToJVM()
        jpype.java.lang.Thread.currentThread().setContextClassLoader(jpype.java.lang.ClassLoader.getSystemClassLoader())
        
    
    conn = jdbc.connect(
                'com.ibm.db2.jcc.DB2Driver',
                config['DB2_CONNECTION_STRING'],
                [config["DB2_USERNAME"], config["DB2_PASSWORD"]],
                'db2jcc4.jar')

    return conn

def fetch_osm_builings(cursor, sql_tablename):
    '''
        This particular function is aimed for obtating all entries from defined rectangle for selected SQL table
    '''

    # fetch column names from defined SQL table

    columns = ['latitude', 'longitude', 'polygon_coordinates', 'vida_confidence']
    
    # sql statement for selecting entries by defined rectangle boundaries
    sql = f"""
        SELECT {', '.join(columns)} FROM USER1.{sql_tablename}
        WHERE 
            (AREA_IN_METERS > {area_threshold}) AND
            (FOOTPRINT_SOURCE = 'osm')
        """
    
    try:
        cursor.execute(sql)
        data = cursor.fetchall()
    except Exception as e:
        print(f"Fetch items error occured: {e}")
        print("Reconnecting to the database try again...")

        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute(sql)
        data = cursor.fetchall()
    finally:
        # reshape obtained data to the GeoDataFrame
        df = pd.DataFrame(data=data, columns=columns)
        df = gpd.GeoDataFrame(df, geometry=shapely.from_wkt(df.polygon_coordinates.astype(str)))
        df = df.drop(['polygon_coordinates'], axis=1)
        df['area_in_meters'] = df["geometry"].apply(lambda g: abs(geod.geometry_area_perimeter(g)[0]))

        return df


def fetch_builings_in_bbox(cursor, lon_min, lon_max, lat_min, lat_max):
    '''
        This particular function is aimed for obtating all entries from defined rectangle for selected SQL table
    '''

    # fetch column names from defined SQL table

    columns = ['latitude', 'longitude', 'polygon_coordinates', 'vida_confidence', 'footprint_source']
    
    # sql statement for selecting entries by defined rectangle boundaries
    sql = f"""
        SELECT {', '.join(columns)} FROM USER1.{sql_tablename}
        WHERE 
            (LATITUDE >= {lat_min}) AND 
            (LATITUDE <= {lat_max}) AND 
            (LONGITUDE >= {lon_min}) AND 
            (LONGITUDE <= {lon_max}) AND
            (AREA_IN_METERS > {area_threshold}) AND
            (FOOTPRINT_SOURCE != 'osm')
        """
    
    try:
        cursor.execute(sql)
        data = cursor.fetchall()
    except Exception as e:
        print(f"Fetch items error occured: {e}")
        print("Reconnecting to the database try again...")

        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute(sql)
        data = cursor.fetchall()
    finally:
        # reshape obtained data to the GeoDataFrame
        df = pd.DataFrame(data=data, columns=columns)
        df = gpd.GeoDataFrame(df, geometry=shapely.from_wkt(df.polygon_coordinates.astype(str)))
        df = df.drop(['polygon_coordinates'], axis=1)
        df['area_in_meters'] = df["geometry"].apply(lambda g: abs(geod.geometry_area_perimeter(g)[0]))

        return df

In [ ]:
# initiate DB connection
conn = connect_to_db()
cursor = conn.cursor()

In [ ]:
# fetch OSM tagged buildings from selected table
df = fetch_osm_builings(cursor, sql_tablename)

In [ ]:
# fetch nonOSM buildings from DB within each OSM buildings bounding box
non_osm_buildings = []

for row in tqdm(df.itertuples(), total=len(df), desc='Fetching buildings'):
    
    # get boundings of building
    min_lon, min_lat, max_lon, max_lat = row.geometry.bounds
    # fetch nonOSM buildings from BD
    buildings = fetch_builings_in_bbox(cursor, min_lon, max_lon, min_lat, max_lat)
    non_osm_buildings.append(buildings)
    

# assemle all collected items to the df
non_osm_buildings_df = pd.concat(non_osm_buildings)

In [ ]:
# filter out only buildings that intersects with OSM building

intersects_building_list = []

for row in tqdm(df.itertuples(), total=len(df), desc='Fetching buildings'):
    
    min_lon, min_lat, max_lon, max_lat = row.geometry.bounds
    
    buildings = non_osm_buildings_df[
                                    (non_osm_buildings_df.longitude >= min_lon) &
                                    (non_osm_buildings_df.longitude <= max_lon) &
                                    (non_osm_buildings_df.latitude >= min_lat) &
                                    (non_osm_buildings_df.latitude <= max_lat)
                                ].copy()
    
    # find buildings that intersects OSM building
    if len(buildings) > 0:
        buildings['under_polygon'] = [row.geometry for i in range(len(buildings))]
        
        buildings['intersects_building'] = [row.geometry.intersects(i.geometry) for i in buildings.itertuples()]
        buildings = buildings[buildings.intersects_building == True]
        buildings = buildings.drop('intersects_building', axis='columns')
        intersects_building_list.append(buildings)

# assemle all collected items to the df
main_df = pd.concat(intersects_building_list)
main_df

In [ ]:
# remove nonOSM entries from the DB

exceptions = []

for row in tqdm(main_df.itertuples(), total=len(main_df)):
    
    try:
        sql = f'''
            DELETE FROM USER1.{sql_tablename} WHERE LATITUDE = {row.latitude} AND LONGITUDE = {row.longitude}; 
                '''
                
        cursor.execute(sql)
    except Exception as e:
        print(f"Entry remove error occured: {e}")
        exceptions.append(row)
        
if len(exceptions) == 0:
    print('All defined buildings were removed from the DB')